In [1]:
import pandas as pd

In [3]:
# Players DataFrame
players_df = pd.read_csv('players_stats.csv')
players_df.rename(columns={'Unnamed: 0': 'id'}, inplace=True) # Rename first column
#players_df

,id,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,0,Álex Abrines,SG,25.0,OKC,31.0,2.0,588.0,56.0,157.0,...,0.923,5.0,43.0,48.0,20.0,17.0,6.0,14.0,53.0,165.0
1,1,Quincy Acy,PF,28.0,PHO,10.0,0.0,123.0,4.0,18.0,...,0.700,3.0,22.0,25.0,8.0,1.0,4.0,4.0,24.0,17.0
2,2,Jaylen Adams,PG,22.0,ATL,34.0,1.0,428.0,38.0,110.0,...,0.778,11.0,49.0,60.0,65.0,14.0,5.0,28.0,45.0,108.0
3,3,Steven Adams,C,25.0,OKC,80.0,80.0,2669.0,481.0,809.0,...,0.500,391.0,369.0,760.0,124.0,117.0,76.0,135.0,204.0,1108.0
4,4,Bam Adebayo,C,21.0,MIA,82.0,28.0,1913.0,280.0,486.0,...,0.735,165.0,432.0,597.0,184.0,71.0,65.0,121.0,203.0,729.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
729,729,Tyler Zeller,C,29.0,MEM,4.0,1.0,82.0,16.0,28.0,...,0.778,9.0,9.0,18.0,3.0,1.0,3.0,4.0,16.0,46.0
730,730,Ante Žižić,C,22.0,CLE,59.0,25.0,1082.0,183.0,331.0,...,0.705,108.0,212.0,320.0,53.0,13.0,22.0,61.0,113.0,459.0
731,731,Ivica Zubac,C,21.0,TOT,59.0,37.0,1040.0,212.0,379.0,...,0.802,115.0,247.0,362.0,63.0,14.0,51.0,70.0,137.0,525.0
732,732,Ivica Zubac,C,21.0,LAL,33.0,12.0,516.0,112.0,193.0,...,0.864,54.0,108.0,162.0,25.0,4.0,27.0,33.0,73.0,281.0


In [5]:
# Teams DataFrame
teams_df = pd.read_csv('team_stats.csv')
teams_df = teams_df.iloc[1:] # Remove first row
teams_df.rename(columns={'Unnamed: 0': 'id'}, inplace=True) # Rename first column
teams_df

,id,Team,Conf,Div,W,L,W/L%,MOV,ORtg,DRtg,NRtg,MOV/A,ORtg/A,DRtg/A,NRtg/A
1,1,Milwaukee Bucks,E,C,60.0,22.0,0.732,8.87,114.23,105.76,8.47,8.05,113.89,106.23,7.66
2,2,Golden State Warriors,W,P,57.0,25.0,0.695,6.46,116.63,110.24,6.39,6.42,116.60,110.24,6.37
3,3,Toronto Raptors,E,A,58.0,24.0,0.707,6.09,113.99,108.00,5.99,5.49,113.78,108.40,5.38
4,4,Utah Jazz,W,NW,50.0,32.0,0.610,5.26,111.35,106.11,5.24,5.29,111.23,105.94,5.28
5,5,Houston Rockets,W,SW,53.0,29.0,0.646,4.77,116.25,111.43,4.82,4.97,116.45,111.43,5.03
6,6,Portland Trail Blazers,W,NW,53.0,29.0,0.646,4.20,115.45,111.07,4.38,4.44,115.63,111.01,4.62
7,7,Denver Nuggets,W,NW,54.0,28.0,0.659,3.95,113.76,109.62,4.14,4.20,113.74,109.35,4.39
8,8,Boston Celtics,E,A,49.0,33.0,0.598,4.44,112.78,108.35,4.43,3.90,112.57,108.69,3.88
9,9,Oklahoma City Thunder,W,NW,49.0,33.0,0.598,3.40,110.85,107.53,3.31,3.56,110.71,107.22,3.49
10,10,Indiana Pacers,E,C,48.0,34.0,0.585,3.33,110.62,107.23,3.39,2.76,110.31,107.50,2.80


In [6]:
# Rookie DataFrame
rookie_df = pd.read_csv('rookie_stats.csv')
rookie_df = rookie_df.iloc[1:] # Remove first row
rookie_df.rename(columns={'Unnamed: 0': 'id'}, inplace=True) # Rename first column
#rookie_df

,id,Player,Debut,Age,Yrs,G,MP,FG,FGA,3P,...,TOV,PF,PTS,FG%,3P%,FT%,MP.1,PTS.1,TRB.1,AST.1
1,1,Jaylen Adams,"Oct 17, '18, ATL @ NYK",22.0,1.0,34.0,428.0,38.0,110.0,25.0,...,28.0,45.0,108.0,0.345,0.338,0.778,12.6,3.2,1.8,1.9
2,2,Deng Adel,"Jan 19, '19, CLE @ DEN",21.0,1.0,19.0,194.0,11.0,36.0,6.0,...,6.0,13.0,32.0,0.306,0.261,1.000,10.2,1.7,1.0,0.3
3,3,DeVaughn Akoon-Purcell,"Oct 23, '18, DEN vs. SAC",25.0,1.0,7.0,22.0,3.0,10.0,0.0,...,2.0,4.0,7.0,0.300,0.000,0.500,3.1,1.0,0.6,0.9
4,4,Rawle Alkins,"Dec 17, '18, CHI @ OKC",21.0,1.0,10.0,120.0,13.0,39.0,3.0,...,8.0,7.0,37.0,0.333,0.250,0.667,12.0,3.7,2.6,1.3
5,5,Grayson Allen,"Oct 22, '18, UTA vs. MEM",23.0,1.0,38.0,416.0,67.0,178.0,32.0,...,33.0,47.0,211.0,0.376,0.323,0.750,10.9,5.6,0.6,0.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111,111,Thomas Welsh,"Oct 17, '18, DEN @ LAC",22.0,1.0,11.0,36.0,7.0,13.0,3.0,...,1.0,3.0,18.0,0.538,0.429,0.500,3.3,1.6,0.4,0.5
112,112,Johnathan Williams,"Oct 22, '18, LAL vs. SAS",23.0,1.0,24.0,372.0,65.0,110.0,0.0,...,16.0,62.0,157.0,0.591,0.000,0.563,15.5,6.5,4.1,0.5
113,113,Kenrich Williams,"Oct 17, '18, NOP @ HOU",24.0,1.0,46.0,1079.0,107.0,279.0,52.0,...,36.0,95.0,279.0,0.384,0.333,0.684,23.5,6.1,4.8,1.8
114,114,Robert Williams,"Oct 22, '18, BOS vs. ORL",21.0,1.0,32.0,283.0,36.0,51.0,0.0,...,10.0,36.0,81.0,0.706,NaN,0.600,8.8,2.5,2.5,0.2
